In [ ]:
import numpy as np
import math
from PIL import Image

In [ ]:
im = Image.open("TestImage3.jpg")
print(im.format, im.size, im.mode)
# get Image

In [ ]:
col, row = im.size
data = np.zeros((row * col, 3))
origin = np.zeros((row, col, 3),'uint8')
pixels = im.load()
for i in range(row):
    for j in range(col):
        r, g, b = pixels[j, i]
        data[col * i + j, :] = r, g, b
        origin[i,j,:]=r, g, b
# get r,g,b of original image

grayscale = np.zeros(row * col)
for i in range(row):
    for j in range(col):
        grayscale[col * i + j] = data[col * i + j, 0] * 0.3 + data[col * i + j, 1] * 0.59 + data[col * i + j, 2] * 0.11
grayscale.resize((row, col))
# get grescale pictures

In [ ]:
im = Image.fromarray(origin,'RGB')
im.show()
# show original image

In [ ]:
im = Image.fromarray(grayscale)
im.show()
# show greysclae image

In [ ]:
sobel = np.zeros((row, col, 2))
magnitude = np.zeros((row, col))
gradient = np.zeros((row, col))
lowmagnitude =np.zeros((row, col))
thita = np.zeros((row, col))
# Gx, Gy, G, thita
maxGrey = 0
minGrey = 9999

#threshold for binary image
#20 for testImage 1,2
#15 for testImage 3
threshold = 15
for i in range(1, row - 1):
    for j in range(1, col - 1):
        sobel[i, j, 0] = grayscale[i - 1, j + 1] + 2 * grayscale[i, j + 1] + grayscale[i + 1, j + 1] \
                         - grayscale[i - 1, j - 1] - 2 * grayscale[i, j - 1] - grayscale[i + 1, j - 1]
        sobel[i, j, 1] = grayscale[i - 1, j - 1] + 2 * grayscale[i - 1, j] + grayscale[i - 1, j + 1] \
                         - grayscale[i + 1, j - 1] - 2 * grayscale[i + 1, j] - grayscale[i + 1, j + 1]
        magnitude[i, j] = (sobel[i, j, 0] ** 2 + sobel[i, j, 1] ** 2) ** 0.5
        if sobel[i, j, 0] == 0:
            thita[i, j] = math.atan(sobel[i, j, 1] / 0.01)
        else:
            thita[i, j] = math.atan(sobel[i, j, 1] / sobel[i, j, 0])
        if magnitude[i, j] > maxGrey:
            maxGrey = magnitude[i, j]
        if magnitude[i, j] < min:
            minGrey = magnitude[i, j]
print maxGrey, minGrey
# use sobel to calculate magnitude

gradient = np.copy(magnitude)
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        magnitude[i, j] = magnitude[i, j] / maxGrey * 255
        gradient[i,j]=magnitude[i,j]
        if magnitude[i, j] > threshold:
            magnitude[i, j] = 255
            count += 1
        else:
            magnitude[i, j] = 0
print count
# get binary image using threshold

# a more accurate image
threshold = 10
for i in range(1, row - 1):
    for j in range(1, col - 1):
        sobel[i, j, 0] = grayscale[i - 1, j + 1] + 2 * grayscale[i, j + 1] + grayscale[i + 1, j + 1] \
                         - grayscale[i - 1, j - 1] - 2 * grayscale[i, j - 1] - grayscale[i + 1, j - 1]
        sobel[i, j, 1] = grayscale[i - 1, j - 1] + 2 * grayscale[i - 1, j] + grayscale[i - 1, j + 1] \
                         - grayscale[i + 1, j - 1] - 2 * grayscale[i + 1, j] - grayscale[i + 1, j + 1]
        lowmagnitude[i, j] = (sobel[i, j, 0] ** 2 + sobel[i, j, 1] ** 2) ** 0.5
        if sobel[i, j, 0] == 0:
            thita[i, j] = math.atan(sobel[i, j, 1] / 0.01)
        else:
            thita[i, j] = math.atan(sobel[i, j, 1] / sobel[i, j, 0])
        if lowmagnitude[i, j] > maxGrey:
            maxGrey = lowmagnitude[i, j]
        if lowmagnitude[i, j] < min:
            minGrey = lowmagnitude[i, j]
print maxGrey, minGrey
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        lowmagnitude[i, j] = lowmagnitude[i, j] / maxGrey * 255
        if lowmagnitude[i, j] > threshold:
            lowmagnitude[i, j] = 255
            count += 1
        else:
            lowmagnitude[i, j] = 0
print count

In [ ]:
im = Image.fromarray(gradient)
if im.mode != 'RGB':
    im = im.convert('RGB')
im.save(str(1)+"gradient.jpg","JPEG")
im.show()
# greyscale done

In [ ]:
whitemagnitude = np.copy(magnitude)
for i in range(row):
    for j in range(col):
        whitemagnitude[i, j] = 255 - whitemagnitude[i,j]
im = Image.fromarray(whitemagnitude)
if im.mode != 'RGB':
    im = im.convert('RGB')
im.save(str(1)+"binary.jpg","JPEG")
im.show()
# greyscale done

In [ ]:
expand_new = np.copy(magnitude)

In [ ]:
# expand original binary image
# don't run for testImage 3!!! 
expand_old = np.copy(magnitude)
flag = 0
for k in range(3):
    for i in range(1, row - 1):
        for j in range(1, col - 1):
            flag = 0
            if expand_old[i-1,j]>0: flag += 1
            if expand_old[i+1,j]>0: flag += 1
            if expand_old[i,j-1]>0: flag += 1
            if expand_old[i,j+1]>0: flag += 1
            if flag>= 2:
                expand_new[i,j] = 255
    expand_old = np.copy(expand_new)

In [ ]:
# image after expand 3 times
im = Image.fromarray(expand_new)
im.show()

In [ ]:
# shrink the image according to the median axis
distance0 = np.copy(expand_new)
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j]>0:
            distance0[i,j]=1
distance_new = np.copy(distance0)
distance_old = np.copy(distance0)
for k in range(15):
    for i in range(1,row-1):
        for j in range(1,col-1):
            min=999
            if distance_old[i-1,j]<min:
                min = distance_old[i-1,j]
            if distance_old[i+1,j]<min:
                min = distance_old[i+1,j]
            if distance_old[i,j-1]<min:
                min = distance_old[i,j-1]
            if distance_old[i,j+1]<min:
                min = distance_old[i,j+1]
            distance_new[i,j]=distance0[i,j]+min
    distance_old = np.copy(distance_new)

for i in range(1,row-1):
    for j in range(1,col-1):
        if distance_old[i,j]<distance_old[i-1,j] or distance_old[i,j]<distance_old[i+1,j] or \
        distance_old[i,j]<distance_old[i,j-1] or distance_old[i,j]<distance_old[i,j+1]:
            distance0[i,j] = 0

#count number of foreground points
count = 0
for i in range(1,row-1):
    for j in range(1,col-1):
        if distance0[i,j] > 0:
            distance0[i,j]=255
            count +=1

In [ ]:
# image after expand and shrink, get rid of many noise
im = Image.fromarray(distance0)
im.show()

In [ ]:
# use a new array to get information about points
point = np.zeros((count, 2))
count = 0
for i in range(1, row - 1):
    for j in range(1, col - 1):
        if distance0[i, j] == 255:
            point[count, :] = i, j
            count += 1
count


In [ ]:
# hough transform, angle:720 partition, p:0-4000 denotes -2000-2000 
angle = 720
p = 4000
hough = np.zeros((angle, p))
for i in range(count):
    if (i % 1000)==0: print i
    for j in range(angle):
        x, y = point[i,:]
        tmpAngle = j / 720.0 * math.pi
        tmpP = x * math.cos(tmpAngle) + y * math.sin(tmpAngle)
        tmpP = int(tmpP) + 2000
        hough[j, tmpP] +=1

In [ ]:
# threshold for effective hough line
# 50 for testImage 1,2
# 105 for testImage 3
threshold = 105
for i in range(angle):
    for j in range(p):
        if hough[i, j] > 100:
            count += 1
print count

tmpHough = np.zeros((count, 3))
tmpCount = 0
for i in range(angle):
    for j in range(p):
        if hough[i, j] > threshold:
            tmpHough[tmpCount,:]=hough[i,j],i,j
            tmpCount += 1
# angleThreshold and pThreshold
# 15,20 both for testImage1,2
# 10,10 for testImage 3
angleThreshold = 10
pThreshold= 10
print tmpCount
effectiveHough = np.zeros((count, 3))
count = 0
for i in range(tmpCount):
    tmp = int(tmpHough[i,0])
    tmpAngle= int(tmpHough[i,1])
    tmpP= int(tmpHough[i,2])
    top = 0 if tmpAngle - angleThreshold < 0 else int(tmpAngle - angleThreshold)
    bottom = angle - 1 if tmpAngle + angleThreshold > angle - 1 else int(tmpAngle + angleThreshold)
    left = 0 if tmpP - pThreshold < 0 else int(tmpP - pThreshold)
    right = p - 1 if tmpP + pThreshold > p - 1 else int(tmpP + pThreshold)
    maxFlag = 1       
    for k in range(top, bottom + 1):
            for l in range(left, right + 1):
                if k==tmpAngle and l==tmpP: continue
                if tmp <= hough[k, l]:
                    maxFlag = 0
                    hough[tmpAngle,tmpP]=0
    if maxFlag == 1:
        effectiveHough[count, :] = tmp, tmpAngle, tmpP
        count+=1
print count
# get effectiveHoughLine array, which contains
# number of edge points on line, anle and p
    
for i in range(count):
    for j in range(i + 1, count):
        if effectiveHough[i, 0] < effectiveHough[j, 0]:
            tmpNum, tmpAngle, tmpP = effectiveHough[i, :]
            effectiveHough[i] = effectiveHough[j]
            effectiveHough[j] = tmpNum, tmpAngle, tmpP

#sort effectiveHough

In [ ]:
# take a look at effectiveHoughLine array
for i in range(count):
    print i, effectiveHough[i]

In [ ]:
# take a look at effectiveHough array, 
# create images show each line on image
lines = np.copy(distance0)
for i in range(27):
    for j in range(row):
        if effectiveHough[i, 1]==0:
            tmpY=int(effectiveHough[i, 2]-2000)
            lines[j, tmpY] = 255
            continue
        tmpY=(effectiveHough[i, 2]-2000)/math.sin(effectiveHough[i, 1] / 720.0 * math.pi)-\
            1/math.tan(effectiveHough[i, 1] / 720.0 * math.pi)*j
        if int(tmpY)>=col or int(tmpY)<0:
            continue
        tmpY=int(tmpY)
        lines[j, tmpY] = 255
    im = Image.fromarray(lines)
    if im.mode != 'RGB':
        im = im.convert('RGB')
    im.save(str(i)+".jpg","JPEG")

### Detect Parallelogram
* pair lines according to their theta angle
* get intersections of paired lines
* compute the parallelogram formed by intersections
* compute frequency that original image's edge points on parallelogram formed by intersections

In [ ]:
# crossPoint function, calculate intersections of 2 given polar
# of 2 given polar coordinates
def crossPoint(angle1,p1,angle2,p2):
    if angle1==0:
        CrossPx=p1
        CrossPy=p2/math.sin(angle2/720.0*math.pi)-1/math.tan(angle2/720.0*math.pi)*CrossPx
        return CrossPx,CrossPy
    if angle2==0:
        CrossPx=p2
        CrossPy=p1/math.sin(angle1/720.0*math.pi)-1/math.tan(angle1/720.0*math.pi)*CrossPx
        return CrossPx,CrossPy
    a = p1/math.sin(angle1/720.0*math.pi)-p2/math.sin(angle2/720.0*math.pi)
    b = 1/math.tan(angle1/720.0*math.pi) - 1/math.tan(angle2/720.0*math.pi)
    CrossPx = a/b
    if CrossPx<0.001: CrossPx=0
    CrossPy = p1/math.sin(angle1/720.0*math.pi)-1/math.tan(angle1/720.0*math.pi)*CrossPx

    return CrossPx,CrossPy

In [ ]:
# pair lines
linePair = np.zeros((10000,2))
pair = 0
for i in range(count):
    for j in range(i+1,count):
        if effectiveHough[i,1]>effectiveHough[j,1]:
            if effectiveHough[i,1]-effectiveHough[j,1]>360:
                angleDif = effectiveHough[j,1]+720-effectiveHough[i,1]
            else:
                angleDif = effectiveHough[i,1]-effectiveHough[j,1]
        else:
            if effectiveHough[j,1]-effectiveHough[i,1]>360:
                angleDif=effectiveHough[i,1]+720-effectiveHough[j,1]
            else:
                angleDif=effectiveHough[j,1]-effectiveHough[i,1]
        if angleDif<40:
            linePair[pair,:]=i,j
            pair+=1
print pair

In [ ]:
# take a look at paired line
for i in range(pair):
    print i,linePair[i]

In [ ]:
from IPython.core.debugger import Tracer

In [ ]:
# get paired lines' intersection 
pointSet = np.zeros((1000000,4,2))
count=0
for i in range(pair):
    for j in range(i+1,pair):
        pair1Line1,pair1Line2=linePair[i,:]
        pair2Line1,pair2Line2=linePair[j,:]
        angle11, angle12 = effectiveHough[int(pair1Line1),1], effectiveHough[int(pair1Line2),1]
        angle21, angle22 = effectiveHough[int(pair2Line1),1], effectiveHough[int(pair2Line2),1]
        if abs(angle11-angle21)<=30 or abs(angle11-angle22)<=30 or \
            abs(angle12-angle21)<=30 or abs(angle12-angle22)<=30:
            continue
        p11,p12=effectiveHough[int(pair1Line1),2]-2000, effectiveHough[int(pair1Line2),2]-2000
        p21,p22=effectiveHough[int(pair2Line1),2]-2000, effectiveHough[int(pair2Line2),2]-2000
        x1,y1=crossPoint(angle11,p11,angle21,p21)
        x2,y2=crossPoint(angle12,p12,angle21,p21)
        x3,y3=crossPoint(angle11,p11,angle22,p22)
        x4,y4=crossPoint(angle12,p12,angle22,p22)
        x5,y5=crossPoint(angle11,p11,angle12,p12)
        x6,y6=crossPoint(angle21,p21,angle22,p22)
        if (i==47 and j==68):
             print count
        if (x1<x5 and x5<x3) or (x3<x5 and x5<x1):
            continue
        if (x2<x6 and x6<x4) or (x4<x6 and x6<x2):
            continue           
        if x1>=row or x1<0:
            continue
        if x2>=row or x2<0:
            continue
        if x3>=row or x3<0:
            continue
        if x4>=row or x4<0:
            continue
        if y1>=col or y1<0:
            continue
        if y2>=col or y2<0:
            continue
        if y3>=col or y3<0:
            continue
        if y4>=col or y4<0:
            continue
        pointSet[count,0:]=x1,y1
        pointSet[count,1:]=x2,y2
        pointSet[count,2:]=x3,y3
        pointSet[count,3:]=x4,y4
        count+=1
        
        
print count

In [ ]:
# PolyArea function, calculate the area of 
# parallelogram using four given coordinates
def PolyArea(x,y):
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

In [ ]:
# a test of PolyArea function
x = [0,100,100,0]
y = [0,0,100,100]
PolyArea(x,y)

In [ ]:
# used to normalize
ratio = 1.0*row/col
ratio

In [ ]:
tmpLine = np.zeros((row, col))
index = 0
validPointSet = np.zeros((10000,5,2))
for  i in range(count):
    final = np.copy(origin)
    
    x1,y1 = pointSet[i,0,:]
    x2,y2 = pointSet[i,1,:]
    x3,y3 = pointSet[i,2,:]
    x4,y4 = pointSet[i,3,:]
    area = PolyArea([x1,x2,x4,x3],[y1,y2,y4,y3])
    
    areaThreshold = 4000
    # Threshold for area, 10000 for testImage1, 2 , 4000 for testImage3
    if area<areaThreshold:
        continue
    # area smaller than 4000, continue, too small
    
    if int(x1)==int(x2):
        k1 = 99999
    else:
        k1 = (y2-y1)/(x2-x1)
    if int(x3)==int(x4): 
        k2 = 99999
    else:
        k2 = (y4-y3)/(x4-x3)
    if int(x1)==int(x3):
        k3 = 99999
    else:
        k3 = (y3-y1)/(x3-x1)
    if int(x4)==int(x2):
        k4 = 99999
    else:
        k4 = (y4-y2)/(x4-x2)
    # some edge condition of k
    
    tmpSumThreshold = 0.58
    sumThreshold = 3.2
    weightSumThreshold = 0.7
    # 3 threshold to determind whether there're
    # enough points on parallelograms' edge
    # 0.3, 2.6, 0.675 for testImage1,2
    # 0.7, 3.5, 0.85 for testImage3
    
    sum = 0
    total1=0
    total2=0
    total3=0
    total4=0
    tmpSum1 = 0
# #######################################
#     if i==1212:
#         Tracer()()
# #######################################

    if x1<=x2:
        xmin=int(x1)
        xmax=int(x2)
    else:
        xmin=int(x2)
        xmax=int(x1)
    if xmax!=xmin:
        for j in range(xmin,xmax):
            tmpY=int((j-x1)*k1+y1)
            if tmpY<0 or tmpY>=col:break
            if lowmagnitude[j,tmpY]==255:
                tmpSum1+=1
        tmpSum1=1.0*tmpSum1/(xmax-xmin)   
    
    tmpSum5 = 0
    if y1<=y2:
        ymin=int(y1)
        ymax=int(y2)
    else:
        ymin=int(y2)
        ymax=int(y1)
    if ymax!=ymin:
        for j in range(ymin,ymax):
            tmpX=int((j-y1)/k1+x1)
            if tmpX-1<0 or tmpX+1>=row:break
            if lowmagnitude[tmpX,j]==255 or lowmagnitude[tmpX-1,j]==255 or lowmagnitude[tmpX+1,j]==255:
                tmpSum5+=1
        tmpSum5=1.0*tmpSum5/(ymax-ymin)   
    tmpSum1=tmpSum1 if abs(x1-x2)>ratio*abs(y1-y2) else tmpSum5
    total1 = abs(x1-x2) if abs(x1-x2)>ratio*abs(y1-y2) else ratio*abs(y1-y2)
    if tmpSum1<tmpSumThreshold: continue
    sum+=tmpSum1
    
    tmpSum2 = 0
    if x3<=x4:
        xmin=int(x3)
        xmax=int(x4)
    else:
        xmin=int(x4)
        xmax=int(x3)
    if xmax!=xmin:
        for j in range(xmin,xmax):
            tmpY=int((j-x3)*k2+y3)
            if tmpY<0 or tmpY>=col:break
            if magnitude[j,tmpY]==255:
                tmpSum2+=1
        tmpSum2=1.0*tmpSum2/(xmax-xmin)  
    
    tmpSum6 = 0
    if y3<=y4:
        ymin=int(y3)
        ymax=int(y4)
    else:
        ymin=int(y4)
        ymax=int(y3)
    if ymax!=ymin:
        for j in range(ymin,ymax):
            tmpX=int((j-y3)/k2+x3)
            if tmpX-1<0 or tmpX+1>=row:break
            if lowmagnitude[tmpX,j]==255 or lowmagnitude[tmpX-1,j]==255 or lowmagnitude[tmpX+1,j]==255:
                tmpSum6+=1
        tmpSum6=1.0*tmpSum6/(ymax-ymin)   
    tmpSum2=tmpSum2 if abs(x3-x4)>ratio*abs(y3-y4) else tmpSum6
    total2 = abs(x4-x3) if abs(x4-x3)>ratio*abs(y4-y3) else ratio*abs(y4-y3)
    if tmpSum2<tmpSumThreshold: continue
    sum+=tmpSum2
    
    tmpSum3 = 0
    if x1<=x3:
        xmin=int(x1)
        xmax=int(x3)
    else:
        xmin=int(x3)
        xmax=int(x1)
    if xmax!=xmin: 
        for j in range(xmin,xmax):
            tmpY=int((j-x1)*k3+y1)
            if tmpY<0 or tmpY>=col:break
            if lowmagnitude[j,tmpY]==255:
                tmpSum3+=1
        tmpSum3=1.0*tmpSum3/(xmax-xmin) 
    
    tmpSum7 = 0
    if y1<=y3:
        ymin=int(y1)
        ymax=int(y3)
    else:
        ymin=int(y3)
        ymax=int(y1)
    if ymax!=ymin: 
        for j in range(ymin,ymax):
            tmpX=int((j-y1)/k3+x1)
            if tmpX-1<0 or tmpX+1>=row:break
            if lowmagnitude[tmpX,j]==255 or lowmagnitude[tmpX-1,j]==255 or lowmagnitude[tmpX+1,j]==255:
                tmpSum7+=1
        tmpSum7=1.0*tmpSum7/(ymax-ymin)   
    tmpSum3=tmpSum3 if abs(x1-x3)>ratio*abs(y1-y3) else tmpSum7
    total3 = abs(x1-x3) if abs(x1-x3)>ratio*abs(y1-y3) else ratio*abs(y1-y3)
    if tmpSum3<tmpSumThreshold: continue
    sum+=tmpSum3
    
    tmpSum4 = 0
    if x2<=x4:
        xmin=int(x2)
        xmax=int(x4)
    else:
        xmin=int(x4)
        xmax=int(x2)
    if xmax!=xmin: 
        for j in range(xmin,xmax):
            tmpY=int((j-x2)*k4+y2)
            if tmpY<0 or tmpY>=col:break
            if lowmagnitude[j,tmpY]==255:
                tmpSum4+=1
        tmpSum4=1.0*tmpSum4/(xmax-xmin)   
 
    tmpSum8 = 0
    if y2<=y4:
        ymin=int(y2)
        ymax=int(y4)
    else:
        ymin=int(y4)
        ymax=int(y2)
    if ymax!=ymin: 
        for j in range(ymin,ymax):
            tmpX=int((j-y2)/k4+x2)
            if tmpX-1<0 or tmpX+1>=row:break
            if lowmagnitude[tmpX,j]==255 or lowmagnitude[tmpX-1,j]==255 or lowmagnitude[tmpX+1,j]==255:
                tmpSum8+=1
        tmpSum8=1.0*tmpSum8/(ymax-ymin)  

    tmpSum4=tmpSum4 if abs(x2-x4)>ratio*abs(y2-y4) else tmpSum8
    total4 = abs(x2-x4) if abs(x2-x4)>ratio*abs(y2-y4) else ratio*abs(y2-y4)
    if tmpSum4<tmpSumThreshold: continue

    sum+=tmpSum4
    weightsum=(total1*tmpSum1+total2*tmpSum2+total3*tmpSum3+total4*tmpSum4)/(total1+total2+total3+total4)
    if sum>sumThreshold and weightsum>weightSumThreshold:
        #Tracer()()
        validPointSet[index,0,:]=x1,y1
        validPointSet[index,1,:]=x2,y2
        validPointSet[index,2,:]=x3,y3
        validPointSet[index,3,:]=x4,y4
        validPointSet[index,4,:]=sum,area
        print index,validPointSet[index],weightsum
        index+=1


In [ ]:
# take a look at valid
for i in range(index):
    print validPointSet[i]

In [ ]:
finalPointSet=np.copy(validPointSet)
for i in range(index):
    for j in range(i+1,index):
        area1=finalPointSet[i,4,1]
        area2=finalPointSet[j,4,1]
        sum1=finalPointSet[i,4,0]
        sum2=finalPointSet[j,4,0]
        x1=0;x2=0;y1=0;y2=0
        for k in range(4):
            x1+=finalPointSet[i,k,0]
            y1+=finalPointSet[i,k,1]
            x2+=finalPointSet[j,k,0]
            y2+=finalPointSet[j,k,1]
        if sum1==0 or sum2==0: continue
        if area1>area2 and area2/area1>0.9 and (abs(x1-x2)<30 and abs(y1-y2)<30):
            if sum1>sum2:
                finalPointSet[j,4,0]=0
                if j==49:print i,j
            else:
                if i==49:print i,j
                finalPointSet[i,4,0]=0
        if area2>area1 and area1/area2>0.9and (abs(x1-x2)<30 and abs(y1-y2)<30):
            if sum1>sum2:
                finalPointSet[j,4,0]=0
            else:
                finalPointSet[i,4,0]=0
for i in range(index):
    if finalPointSet[i,4,0]>0:
        print finalPointSet[i]

In [ ]:
#final = np.copy(origin)
for i in range(index):
    if finalPointSet[i,4,0]>-1:
        final = np.copy(origin)

        x1,y1=validPointSet[i,0,:]
        x2,y2=validPointSet[i,1,:]
        x3,y3=validPointSet[i,2,:]
        x4,y4=validPointSet[i,3,:]
        
        
        k1 = (y2-y1)/(x2-x1)
        k2 = (y4-y3)/(x4-x3)
        k3 = (y3-y1)/(x3-x1)
        k4 = (y4-y2)/(x4-x2)
        if x1<=x2:
            xmin=int(x1)
            xmax=int(x2)
        else:
            xmin=int(x2)
            xmax=int(x1)
        for j in range(xmin,xmax):
            tmpY=int((j-x1)*k1+y1)
            final[j,tmpY,:]=255,0,0
        
        if x3<=x4:
            xmin=int(x3)
            xmax=int(x4)
        else:
            xmin=int(x4)
            xmax=int(x3)
        for j in range(xmin,xmax):
            tmpY=int((j-x3)*k2+y3)
            final[j,tmpY,:]=255,0,0
                
        if x1<=x3:
            xmin=int(x1)
            xmax=int(x3)
        else:
            xmin=int(x3)
            xmax=int(x1)
        for j in range(xmin,xmax):
            tmpY=int((j-x1)*k3+y1)
            final[j,tmpY,:]=255,0,0
                
        if x2<=x4:
            xmin=int(x2)
            xmax=int(x4)
        else:
            xmin=int(x4)
            xmax=int(x2)
        for j in range(xmin,xmax):
            tmpY=int((j-x2)*k4+y2)
            final[j,tmpY,:]=255,0,0
            
            
        if y1<=y2:
            ymin=int(y1)
            ymax=int(y2)
        else:
            ymin=int(y2)
            ymax=int(y1)
        if ymax!=ymin:
            for j in range(ymin,ymax):
                tmpX=int((j-y1)/k1+x1)
                if tmpX<0 or tmpX>=row: break
                final[tmpX,j,:]=255,0,0
        
        if y3<=y4:
            ymin=int(y3)
            ymax=int(y4)
        else:
            ymin=int(y4)
            ymax=int(y3)
        if ymax!=ymin:
            for j in range(ymin,ymax):
                tmpX=int((j-y3)/k2+x3)
                if tmpX<0 or tmpX>=row: break
                final[tmpX,j,:]=255,0,0
                
        if y1<=y3:
            ymin=int(y1)
            ymax=int(y3)
        else:
            ymin=int(y3)
            ymax=int(y1)
        if ymax!=ymin:
            for j in range(ymin,ymax):
                tmpX=int((j-y1)/k3+x1)
                if tmpX<0 or tmpX>=row: break
                final[tmpX,j,:]=255,0,0
        
        if y2<=y4:
            ymin=int(y2)
            ymax=int(y4)
        else:
            ymin=int(y4)
            ymax=int(y2)
        if ymax!=ymin:
            for j in range(ymin,ymax):
                tmpX=int((j-y2)/k4+x2)
                if tmpX<0 or tmpX>=row: break
                final[tmpX,j,:]=255,0,0
        
        im = Image.fromarray(final,'RGB')
        if im.mode != 'RGB':
            im = im.convert('RGB')
        im.save(str(i)+"result.jpg","JPEG")
        #im.show() 